In [15]:
%pip install -q neo4j

In [16]:
from google.colab import userdata
NEO4J_URI = userdata.get('NEO4J_URI')
NEO4J_USER = userdata.get('NEO4J_USERNAME')
NEO4J_PWD = userdata.get('NEO4J_PASSWORD')

In [19]:
from neo4j import GraphDatabase

class Neo4jGraph:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_graph(self):
        with self.driver.session() as session:
            # Create the legend node
            session.run("""
                MERGE (legend:Legend {name: 'Legend'})
                SET legend.summary = 'Legend: Blue for sections, Green for subsections, Red for the legend',
                    legend.color = 'red'
            """)

            # Add sections
            sections = [
                ("Abstract", "Overview of GLiREL's novel approach for zero-shot relation classification."),
                ("1 Introduction", "Motivation and importance of efficient zero-shot relation extraction."),
                ("2 Background", "Discussion of prior work, including NER and zero-shot techniques."),
                ("3 Method", "Details of GLiREL's architecture and training methodology."),
                ("4 Experiments", "Empirical evaluation using benchmark datasets and results."),
                ("5 Analysis", "Performance analysis, including inference speed and ablation studies."),
                ("6 Conclusion", "Summary of findings and GLiREL's contributions."),
                ("7 Limitations", "Discussion of limitations and future directions."),
                ("References", "List of cited works and related research."),
                ("Appendix", "Supplementary material, including extended results and technical details.")
            ]

            for name, summary in sections:
                session.run("""
                    MERGE (section:Section {name: $name})
                    SET section.summary = $summary, section.color = 'blue'
                """, name=name, summary=summary)

            # Add edges for section flow
            for i in range(len(sections) - 1):
                session.run("""
                    MATCH (from:Section {name: $from_name}), (to:Section {name: $to_name})
                    MERGE (from)-[:TRANSITIONS_TO {summary: $transition_summary}]->(to)
                """, from_name=sections[i][0], to_name=sections[i + 1][0],
                   transition_summary=f"Transition from {sections[i][0]} to {sections[i + 1][0]}.")

            # Add subsections and their edges
            subsections = {
                "2 Background": [
                    ("Joint vs Independent NER and Relation Classification",
                     "Compares joint and independent approaches to NER and relation classification."),
                    ("Zero-Shot Relation Extraction", "Reviews existing zero-shot relation extraction models."),
                    ("LLMs for Relation Classification", "Explores the use of LLMs for relation classification."),
                    ("Zero-Shot Learning and Synthetic Training Data Generation",
                     "Highlights synthetic data generation for training zero-shot models.")
                ],
                "3 Method": [
                    ("Input", "Details the input structure, including relation labels, text tokens, and entity indices."),
                    ("Token Representation", "Explains how tokens are encoded and represented in the model."),
                    ("Label and Entity Pair Representation", "Describes how relation labels and entity pairs are encoded."),
                    ("Refinement Layer", "Explains the refinement process using attention mechanisms."),
                    ("Scoring Layer", "Details the scoring mechanism for relationships between entity pairs and relation types."),
                    ("Training Dataset Generation", "Outlines the protocol for generating synthetic training data."),
                    ("Extending to Coreference Resolution and Document-Level Relation Classification",
                     "Discusses extending GLiREL for document-level relation classification.")
                ]
                # Add more subsections as required
            }

            for section, subs in subsections.items():
                for sub_name, sub_summary in subs:
                    session.run("""
                        MATCH (parent:Section {name: $section_name})
                        MERGE (sub:Subsection {name: $sub_name})
                        SET sub.summary = $sub_summary, sub.color = 'green'
                        MERGE (parent)-[:HAS_SUBSECTION]->(sub)
                    """, section_name=section, sub_name=sub_name, sub_summary=sub_summary)


In [20]:
# Connect to Neo4j
uri = NEO4J_URI       # Update with your Neo4j URI
user = NEO4J_USER     # Update with your Neo4j username
password = NEO4J_PWD  # Update with your Neo4j password

neo4j_graph = Neo4jGraph(uri, user, password)
neo4j_graph.create_graph()
neo4j_graph.close()